In [5]:
require "hdf5"
require "optim"
require "nn"

In [6]:
f = hdf5.open("data.hdf5", "r")

X_train = f:read("X_train"):all()
Y_train = f:read("Y_train"):all()
X_valid = f:read("X_valid"):all()
Y_valid = f:read("Y_valid"):all()
X_test = f:read("X_test"):all()
nwords = f:read("nwords"):all()[1]
nclasses = f:read("nclasses"):all()[1]


--sentences
X_valid_sen = f:read("X_valid_sen"):all()
X_test_sen = f:read("X_test_sen"):all()
X_train_sen = f:read("X_train_sen"):all()
--MEMM
X_train_MEMM = f:read("X_train_MEMM"):all()
X_valid_MEMM = f:read("X_valid_MEMM"):all()
X_valid_sen_MEMM = f:read("X_valid_sen_MEMM"):all()
nfeaturesMEMM = f:read("nfeaturesMEMM"):all()[1]

C = nclasses

/Users/Vincent/.luarocks/share/lua/5.1/hdf5/group.lua:312: HDF5Group:read() - no such child 'X_train_sen' for [HDF5Group 33554434 /]
stack traceback:
	[C]: in function 'error'
	/Users/Vincent/.luarocks/share/lua/5.1/hdf5/group.lua:312: in function 'read'
	[string "f = hdf5.open("data.hdf5", "r")..."]:15: in main chunk
	[C]: in function 'xpcall'
	/Users/Vincent/torch/install/share/lua/5.1/itorch/main.lua:179: in function </Users/Vincent/torch/install/share/lua/5.1/itorch/main.lua:143>
	/Users/Vincent/torch/install/share/lua/5.1/lzmq/poller.lua:75: in function 'poll'
	...s/Vincent/torch/install/share/lua/5.1/lzmq/impl/loop.lua:307: in function 'poll'
	...s/Vincent/torch/install/share/lua/5.1/lzmq/impl/loop.lua:325: in function 'sleep_ex'
	...s/Vincent/torch/install/share/lua/5.1/lzmq/impl/loop.lua:370: in function 'start'
	/Users/Vincent/torch/install/share/lua/5.1/itorch/main.lua:350: in main chunk
	[C]: in function 'require'
	(command line):1: in main chunk
	[C]: at 0x0105f33bd0: 

In [3]:
function process(inputs, numwords)
    processed = inputs:clone()
    for i = 1, inputs:size(1) do
        processed[i][2] = inputs[i][2] + numwords
    end
    return processed
end

for i = 1,X_valid_sen_MEMM:size(1) do
    X_valid_sen_MEMM[i] = process(X_valid_sen_MEMM[i], nwords)
end

Y_valid_sen = format_valid(Y_valid, X_valid_sen)
y = Y_train

In [11]:
addB = nn.Add(nclasses)
lookup = nn.LookupTable(nwords+nclasses,nclasses)
sum = nn.Sum(2)
tanh = nn.Tanh()
linear = nn.Linear(nclasses, nclasses)
softmax = nn.LogSoftMax()
h = nn.Sequential()
h:add(lookup)
h:add(sum)
h:add(addB)


w, dl_dw = h:getParameters()
w:zero()

In [14]:
function gen_score(curr_class, prev_class, input, nword)
    local input1 = input[1]
    local pair = torch.Tensor{{input1, prev_class+nword}}
    local answer = (h:forward(pair))
    --answer = nn.LogSoftMax():forward(answer)
    return answer[1][curr_class]
end

function viterbi_gen(sequence, scoring_function, nclass, nword)
    local P_table = torch.ones(nclass, sequence:size(1)):mul(-1e10)
    local BP = torch.zeros(nclass, sequence:size(1))
    local initial = torch.ones(nclass,1) :mul(-1e10)
    initial[8] = 1
    P_table[{{},{1,1}}] = initial
    
    local max_ind = 0
    local max_prob = -1e10
    for i = 2,sequence:size(1) do
        for curr_class_ind = 1,nclass do
            for prev_class_ind = 1,nclass do
                local best_available_P = P_table[curr_class_ind][i]
                local candidate_P = scoring_function(curr_class_ind, prev_class_ind, sequence[i], nword)
                if candidate_P + P_table[prev_class_ind][i-1] > best_available_P then
                    P_table[curr_class_ind][i] = candidate_P + P_table[prev_class_ind][i-1]
                    BP[curr_class_ind][i] = prev_class_ind
                    if P_table[curr_class_ind][i] > max_prob then
                        max_prob = P_table[curr_class_ind][i]
                        max_ind = curr_class_ind
                    end
                end
            end
        end
    end
    
    local classes = torch.zeros(sequence:size(1))
    classes[sequence:size(1)] = max_ind
    for i = sequence:size(1), 2, -1 do
        classes[i-1] = BP[classes[i]][i]
    end
    return classes
end

function format_valid(valid, valid_sen)
    local y_tags = torch.zeros(valid_sen:size(1), valid_sen:size(2))
    local sen_idx = 0
    local col_idx = 0
    for i=1,valid:size(1) do
        if valid[i] == 8 then
            sen_idx = sen_idx + 1
            col_idx = 1
        end
        y_tags[sen_idx][col_idx] = valid[i]
        col_idx = col_idx + 1
    end
    return y_tags
end

function compare(preds, targets)
    
    local indices = torch.zeros(preds:size(1))
    local int = 0
    for i = 1, preds:size(1) do
        if preds[i] ~= targets[i] then
            int = int + 1
            indices[int] = i
        end
    end
    if int == 0 then
        return 1
    end
    return indices:sub(1,int)
end
        
function update(sequence, preds, targets, nclass, nword, eta)
    
    local indexes = compare(preds,targets)
    
    if indexes == 1 then
        return
    end
    for i = 1,indexes:size(1) do
        local index = indexes[i]
        if index == 1 then
           
            input = torch.Tensor{{sequence[index][1], 9+nword}}
        else
          
            input = torch.Tensor{{sequence[index][1], targets[index-1]+nword}}
        end
        
        local prediction = h:forward(input):t()
        
        
        local out_grad = torch.zeros(nclass)
        out_grad[preds[index]]= 1/(prediction[preds[index]])[1]
        out_grad[targets[index]] = -1/(prediction[targets[index]])[1]
       
        h:zeroGradParameters()
        h:backward(input[1], out_grad)
        h:updateParameters(eta)
    end
end

function train_sen(sequence, target, nclass, nword, eta)
    local preds = viterbi_gen(sequence, gen_score, nclass, nword)
    update(sequence, preds, target, nclass, nword, eta)
   
    --print(preds)
    --print(targets)
    --print(compare(preds,targets):size(1))
    z = compare(preds,target)
    if z == 1 then
        return 0
    end
    return z:size(1)
end

function train_corp(data, targs, nclass,nword,eta,nepochs)
    for j = 1,nepochs do
        idx = torch.randperm(data:size(1))
        
        for k = 1,data:size(1) do
            i = idx[k]
            local sen = data[i]:sub(1,torch.nonzero(data[{{i,i},{},{1,1}}]):size(1))
            local length = train_sen(sen, targs[i]:sub(1,sen:size(1)), nclass, nword, eta)
            
            if k % 100 == 0 then
                print(k,length/sen:size(1))
            end
        end
    end
end
    

In [15]:
Y_train_sen = format_valid(Y_train, X_train_sen)

In [27]:
--RUN THIS TO TRAIN, ONE EPOCH TAKES 10 MIN
train_corp(X_train_sen, Y_train_sen, nclasses, nwords, 1, 1)

100	0.38461538461538	


200	0.5	


300	0.33333333333333	


400	0.12903225806452	


500	0.090909090909091	


...

2900	0.033333333333333	


3000	0.15	


3100	0.068965517241379	


3200	0.35294117647059	


3300	0.33333333333333	


Columns 1 to 26
 8  5  5  5  5  5  5  5  5  5  5  5  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 8  3  3  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 8  3  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
 8  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
 8  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
 8  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  0
 8  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
 8  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
 8  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
 8  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1

Columns 27 to 52
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 1  1  1  1  1  1  1  1  1  1 

In [20]:
function predict_tags(X)
    local predictions = torch.zeros(X:size(1), X:size(2))

    for i=1,X:size(1) do
        local sen = (X[i]:sub(1,torch.nonzero(X[{{i,i},{},{1,1}}]):size(1)))
        
        
        local p = viterbi_gen(sen, gen_score, C, nwords)
       
        predictions[{{i,i},{1,p:size(1)}}] = p
    end
    return predictions
end

function predict_fscore(p_tags, y_tags)
    local rel_retrived = {[1]=0,[2]=0,[3]=0,[4]=0,[5]=0}--,[6]=0,[7]=0}
    local rel_notretrived = {[1]=0,[2]=0,[3]=0,[4]=0,[5]=0}--,[6]=0,[7]=0}
    local irrel_retrived = {[1]=0,[2]=0,[3]=0,[4]=0,[5]=0}--,[6]=0,[7]=0}
    for i=1,p_tags:size(1) do
        for j=1,p_tags:size(2) do
            for tag=1,5 do        
                if y_tags[i][j] == tag then
                    rel_notretrived[tag] = rel_notretrived[tag] + 1
                    if p_tags[i][j] == tag then
                        rel_retrived[tag] = rel_retrived[tag] + 1
                    else
                        irrel_retrived[tag] = irrel_retrived[tag] + 1
                    end
                end
            end
        end
    end
    local recall = 0
    local precis = 0
    for i=1,5 do
        local r = (rel_retrived[i]/(rel_retrived[i]+rel_notretrived[i]))
        local p = (rel_retrived[i]/(rel_retrived[i]+irrel_retrived[i]))
        --print(i, p, r, 2*p*r/(p+r))
        recall = recall + r
        precis = precis + p
    end
    return recall/5,precis/5
end

-- Writes to file in Kaggle prediction format
-- p : prediction tensor
-- fname : output filename

function kaggle_format(p, fname)
    local f = io.open("predictions/" .. fname, "w")
    f:write("ID,Labels\n")
    for i=1,p:size(1) do
        local s = ""
        local t = {[1]="O",[2]="PER",[3]="LOC",[4]="ORG",[5]="MISC",[6]="MISC",[7]="LOC",[8]="O",[9]="O"}
        local prev_tag = 0
        local prev_idx = 0
        for j=2,p:size(2) do
            if p[i][j] == 0 then
                break
            end
            if p[i][j] ~= 1 then
                if prev_tag == 0 then
                    s = s .. t[p[i][j]] .. "-" .. tostring(j-1)
                elseif p[i][j] == prev_tag and prev_idx == j-1 then
                    s = s .. "-" .. tostring(j-1) 
                else
                    s = s .. " " .. t[p[i][j]] .. "-" .. tostring(j-1)
                end
                prev_tag = p[i][j]
                prev_idx = j
            end
        end
        f:write(i .. "," .. s .. "\n")
    end
    f:close()
end

In [ ]:
--TAKES A FEW MINUTES TO RUN
preds = predict_tags(X_test_sen)
kaggle_format(preds, "pred_" .. tostring(os.time()) .. "_memm1.csv")

In [29]:
--TAKES A FEW MINUTES TO RUN
preds = predict_tags(X_valid_sen)
r, p = predict_fscore(preds, Y_valid_sen)
print(r, p)
print(2*r*p/(r+p))


0.1474029143821	0.24554232441313	
0.18421729365456	
